In [1]:
from plotDensity import *
import mpmath as mp

In [2]:
def log_sum_exp(to_sum):
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value-maxval)
    res = maxval + mp.log(exp_sum)
    return res

Accuracy is set to e-10

In [3]:
X, Y = (4,4)

"""Incoming data to get handled by new result handling function"""
interval_data = [
  627.1095947266,
  628.8754638672,
  630.9088623047,
  632.5491699219,
  633.5110107422,
  404.7702880859,
  406.3506591797,
  407.3104492188,
  407.8216796875,
  407.8045410156,
  404.0828857422,
  405.1276611328,
  405.6312011719,
  405.6431152344,
  405.1499023438,
  401.4141113281,
  401.9133789062,
  401.9149902344,
  401.4306152344,
  400.5290527344,
  402.1429199219,
  402.1175537109,
  401.6156250000,
  400.6624267578,
  399.0767089844,
  613.2126708984,
  612.2446777344,
  610.6416259766,
  608.6352050781,
  606.9015625000
]

"""data after applying min rescaling in new result handling function"""
min_rescaled_data = [
  0.0000000000,
  1.7658691406,
  3.7992675781,
  5.4395751953,
  6.4014160156,
  0.0000000000,
  1.5803710938,
  2.5401611328,
  3.0513916016,
  3.0342529297,
  0.0000000000,
  1.0447753906,
  1.5483154297,
  1.5602294922,
  1.0670166016,
  0.8850585938,
  1.3843261719,
  1.3859375000,
  0.9015625000,
  0.0000000000,
  3.0662109375,
  3.0408447266,
  2.5389160156,
  1.5857177734,
  0.0000000000,
  6.3111083984,
  5.3431152344,
  3.7400634766,
  1.7336425781,
  0.0000000000
]

"""data after rescaling for stitching intervals"""
stitched_data = [
  0.0000000000,
  1.7658691406,
  3.7992675781,
  5.4395751953,
  6.4014160156,
  3.8592041016,
  5.4395751953,
  6.3993652344,
  6.9105957031,
  6.8934570313,
  5.3545898438,
  6.3993652344,
  6.9029052734,
  6.9148193359,
  6.4216064453,
  6.3993652344,
  6.8986328125,
  6.9002441406,
  6.4158691406,
  5.5143066406,
  6.9256103516,
  6.9002441406,
  6.3983154297,
  5.4451171875,
  3.8593994141,
  6.3983154297,
  5.4303222656,
  3.8272705078,
  1.8208496094,
  0.0872070313
]

cut_data = {
  "-22": 0.0000000000,
  "-18": 1.7658691406,
  "-14": 3.7992675781,
  "-10": 5.4395751953,
  "-6": 6.3993652344,
  "-2": 6.8986328125,
  "2": 6.9002441406,
  "6": 6.3983154297,
  "10": 5.4303222656,
  "14": 3.8272705078,
  "18": 1.8208496094,
  "22": 0.0872070312
}

normalized_data = {
  "-22": 2.8611055117,
  "-18": 4.6269746524,
  "-14": 6.6603730899,
  "-10": 8.3006807071,
  "-6": 9.2604707461,
  "-2": 9.7597383242,
  "2": 9.7613496524,
  "6": 9.2594209414,
  "10": 8.2914277774,
  "14": 6.6883760196,
  "18": 4.6819551211,
  "22": 2.9483125430
}


In [4]:
old_result_handling_out = read_data_from_file("../results/test/test_intervals6.txt")

"""Reducing to single walker resutl per interval"""
walker_results = average_matching_keys(old_result_handling_out)

In [5]:
old_result_handling_out_concatenated = [list(d.values()) for d in walker_results]

DEVIATION INPUT RESULTS

In [6]:
max(abs((np.concatenate(old_result_handling_out_concatenated)) - np.array(interval_data)))

0.0

In [7]:
"""minimum rescaling of results per interval"""
old_result_handling_min_rescaled =  get_renormalized_log_g_values(walker_results)

DEVIATION MIN RESCALE

In [8]:
max(abs(np.concatenate(old_result_handling_min_rescaled[1])- np.array(min_rescaled_data)))

1.0003353700938078e-10

In [9]:
walker_results = old_result_handling_out

"""normalize the walker results by min value for log results"""
walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)

"""averages over walker results per intervals"""
walker_results = average_matching_keys(walker_results)

results_x = []
results_y = []
for result in walker_results:
    results_y.append(np.array(list(result.values())))
    results_x.append(np.array(list(result.keys())))

derivatives_wrt_e = get_derivative_wrt_e(walker_results)
minimum_deviation_energies = find_lowest_inverse_temp_deviation(derivatives_wrt_e)
rescale_results_for_concatenation(results_x, results_y, minimum_deviation_energies)

In [10]:
"""Store concatenate interval results"""
concatenated_keys = np.concatenate(results_x)
concatenated_values = np.concatenate(results_y)
list_of_concat_rescale_dicts = []
for keys, values in zip(results_x, results_y):
    # Combine keys and values into a dictionary
    dict_from_arrays = {k: v for k, v in zip(keys, values)}
    list_of_concat_rescale_dicts.append(dict_from_arrays)

DEVIATION CONCATENATE

In [11]:
max(abs(concatenated_values - np.array(stitched_data)))

2.0010038070950031e-10

In [12]:
"""Cut overlapping parts"""
x_max = -1 -2*X*Y
unique_x = []
unique_y = []

sorted_dict_list = [dict(sorted(d.items())) for d in list_of_concat_rescale_dicts]

for dictionary in sorted_dict_list:
    for key, value in dictionary.items():
        if key > x_max: #avoid double counting
            x_max = key
            unique_x.append(key)
            unique_y.append(value)


DEVIATION CUT OVERLAP

In [13]:
np.array(unique_y) - np.array(list(cut_data.values()))

array([ 0.00000000e+00, -1.11022302e-14,  5.72875081e-14, -5.32907052e-15,
        2.05078120e-03,  1.19628906e-02, -6.78710940e-03,  2.32910157e-02,
        8.39843751e-02,  3.21289064e-02,  9.99873517e-11,  1.99998421e-10])

THIS IMPLIES PROBLEM WITH NAIVE CUT OF OVERLAP IN PYTHON

ASSUMING NOW INPUT FOR LOG SUM EXP RESCALING WHICH GOES INTO C CODE

In [14]:
cut_data_input = np.array(list(cut_data.values()))

In [15]:
offset = log_sum_exp(cut_data_input)
rescaled_y = [res + mp.log(2)*16 - offset for res in cut_data_input]
print(rescaled_y)

[mpf('2.8611055117440856'), mpf('4.6269746523440851'), mpf('6.6603730898440858'), mpf('8.3006807070440853'), mpf('9.2604707461440849'), mpf('9.7597383242440845'), mpf('9.7613496523440855'), mpf('9.2594209414440858'), mpf('8.2914277773440848'), mpf('6.6883760195440853'), mpf('4.6819551211440853'), mpf('2.9483125429440857')]


DEVIATION LOG SUM EXP RESCALING FOR SAME INPUT

In [16]:
max(abs(np.array(rescaled_y) - np.array(list(normalized_data.values()))))

mpf('5.5915272412221384e-11')

ALTERNATIVE WAY OF CUTTING THE OVERLAP ANALOGOUSLY TO C CODE

In [17]:
def cut_overlapping_histogram_parts(interval_data, stitching_keys):
    for i in range(len(stitching_keys)):
        stitching_energy_of_interval_i = stitching_keys[i]

        # Modify the i-th interval
        current_interval = interval_data[i]
        # Keep only keys <= stitching_energy_of_interval_i
        current_interval = {k: v for k, v in current_interval.items() if k <= stitching_energy_of_interval_i}

        # Modify the (i+1)-th interval if following interval is still in bounds
        if i + 1 < len(interval_data):
            next_interval = interval_data[i + 1]
            # Keep only keys > stitching_energy_of_interval_i
            next_interval = {k: v for k, v in next_interval.items() if k > stitching_energy_of_interval_i}

        # Update the intervals in the original list
        interval_data[i] = current_interval
        if i + 1 < len(interval_data):
            interval_data[i + 1] = next_interval

In [18]:
cut_overlapping_histogram_parts(list_of_concat_rescale_dicts, minimum_deviation_energies)

In [20]:
# Flatten the list of dictionaries into a single list of values
cut_values = [value for d in list_of_concat_rescale_dicts for value in d.values()]

# Convert the list of values into a NumPy array
cut_values_array = np.array(cut_values)


DEVIATION OF CUT RESULTS NEW IMPLEMENTATION

In [21]:
max(abs(cut_values_array-np.array(list(cut_data.values()))))

2.0010038070950031e-10